In [ ]:
import gzip
import datetime
import numpy as np
import theano
import theano.tensor as T

stock='mnist.pkl.gz'
x=stock['x']
y=stock['y']

In [ ]:
batch_size = k  ## 64*sth
num_layer = l   ## l+2 together if we add input and output layer
num_neutron = m ## l*1 vector, each denotes the number of neutrons in the layer
eta = 0.1 # Learning rate
lambdas = 0.001 # regularization factor 
x = T.dmatrix('x')
y = T.lvector('y')
n_in_0 = ncol(x)
n_out_0 = num_neutron[0]
n_in_1 = n_out_0
n_out_1 = num_neutron[1]
## ... so on so forth
n_out_l = 1


## Below are the initial values
w01 = theano.shared(np.random.uniform(low=-np.sqrt(6. / (n_in_0 + n_out_0)),
     high=np.sqrt(6. / (n_in_0 + n_out_0)),size=(n_in_0, n_out_0)),name='w01',borrow=True)
w12 = theano.shared(np.random.randn(n_in_1,n_out_1),name='w12',borrow=True)
w23 = theano.shared(np.random.randn(n_in_2,n_out_2),name='w23',borrow=True)
##...more w corresponding to l
b01 = theano.shared(value=np.zeros(n_out_0, dtype=theano.config.floatX),name='b1',borrow=True)
b12 = theano.shared(value=np.zeros(n_out_1, dtype=theano.config.floatX),name='b2',borrow=True)
b23 = theano.shared(value=np.zeros(n_out_2, dtype=theano.config.floatX),name='b3',borrow=True)
##...more b corresponding to w


#########################################################
############# From MPI recieve new w and b ##############
#########################################################
w01 = theano.shared(new_w01,name='w01',borrow=True)
# ......

L01 = T.tanh(T.dot(x, w01) + b01) ## standard first step transformation
L12 = T.dot(L01, w12) + b12 ## linear regression here
L23 = T.dot(L12, w23) + b23
## ...more layers
## L(l+1) is the output layer, which is just a nrow*1 vector in our case
    
loss = T.mean((y-Lout)**2)  
cost = loss.mean() + lambdas * ((w01**2).sum() + (w12**2).sum()+....)  ## L2 penalty
gw01 = T.grad(cost=cost, wrt=w01)
gw12 = T.grad(cost=cost, wrt=w12)
#...
gb01 = T.grad(cost=cost, wrt=b01)
gb12 = T.grad(cost=cost, wrt=b12)
#...
train = theano.function(inputs=[x,y], outputs=[Lout,loss,w,b],
        updates=((w01, w01 - eta * gw01), (w12, w12 - eta * gw12),
        (b01, b01 - eta * gb01), (b12, b12 - eta * gb12)),name='train')
test = theano.function(inputs=[x,y],outputs=[Lout, loss],name='test')

## openmp here
Lout_par=np.zeros(64,n)
loss_par=np.zeros(64)
w_par=np.zeros(64, dim(w01)+dim(w12)+....) # dim means n_in*n_out
b_par=np.zeros(64,len(b01)+len(b12)+....)
for i in prange(64):  ## 64 threads, 64 parallel updates, each updates has 10 minibatch sub-updates
    ## sequential below:
    for j in range(num_batch): ## for example, data size=10000, number of batch here is 10, and batch size=5
        [Lout_par[i,],loss_par[i,],w[i,],b[i,]]=train([x,y])

w_new=np.mean(w_par,axis=0)
b_new=np.mean(b_par.axis=0)
dw=w_new-w_old
db=b_new-b_old
## send it to parameter server